# For Comparison, Training the Same Model With CPU

First we have to change backend to CPU using plaidML

To use PlaidML as backend follow these steps:
- activate project environment
- `plaidml-setup`
- `Enable experimental device support? (y,n)[n]:n`
- `Please choose a default device: 1` option 1 is always the CPU
- `Save settings to /Users/veersingh/.plaidml? (y,n)[y]:y`

In [ ]:
# import libraries and set backend to plaidML
from os import environ
environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.preprocessing.image import ImageDataGenerator
import time
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import json
import PIL

In [ ]:
# Specify the paths
train_dataset_path = '/Users/veersingh/Desktop/cats_dogs/train'
valid_dataset_path = '/Users/veersingh/Desktop/cats_dogs/valid'

In [ ]:
# Rescale
train = ImageDataGenerator(rescale=1 / 255)
valid = ImageDataGenerator(rescale=1 / 255)

# Define train and valid dataset
# This method is useful since our images are in their respective class folders

# seed = random number to maintain same output everytime
# directory = path of train or valid dataset
# target_size = resize the input image to required shape
# batch_size = default batch size of 32
# class_mode = categorical for multiclass
# color_mode = depending on number of channels required by model, grayscale for 1 channel, rgb for 3 channel

train_dataset = train.flow_from_directory(seed=1,
                                          directory=train_dataset_path,
                                          target_size=(100, 100),
                                          batch_size=32,
                                          class_mode = 'binary',
                                          color_mode='rgb')

valid_dataset = valid.flow_from_directory(seed=1,
                                         directory=valid_dataset_path,
                                         target_size=(100, 100),
                                         batch_size=32,
                                         class_mode = 'binary',
                                         color_mode='rgb')

# print labels
print('\n')
print(f'Label for cat -> {train_dataset.class_indices["cat"]}')
print(f'Label for dog -> {train_dataset.class_indices["dog"]}')

In [ ]:
# Model
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(filters=32, kernel_size=(3, 3),activation='relu',input_shape=(100, 100, 3)))

# Maxpool layer 1
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

# Maxpool layer 2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))

# Maxpool layer 3
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 4
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))

# Maxpool layer 4
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten to 1D
model.add(Flatten())

# Fully connected layer with 512 neurons
model.add(Dense(units=512, activation='relu'))

# Output layer with single neuron which gives 0 for cat and 1 for dog
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Train the model

# start the timer
start_time = time.time()

# generator = input training data created using ImageDataGenerator
# steps_per_epoch = number of total training images/batch size = 16000/32 = 500
# epochs = number of times we go through the entire training dataset
# validation_data = validation dataset
# verbose = it will print updates on every epoch

model.fit_generator(generator=train_dataset,
                              steps_per_epoch=500,
                              epochs=30,
                              validation_data=valid_dataset,
                              verbose=2)

# save the model
model.save('models/cats_dogs_trained_model.h5')

# stop the timer
time_elapsed = round(time.time()-start_time, 2)

print(f'\n\nThe model took {time_elapsed} seconds to train' )

## Save time taken to JSON

In [ ]:
log = {"time_to_train_cpu":time_elapsed}

with open("log_cpu.json", "w") as data_file:
    json.dump(log, data_file, indent=4)